# Object Relational Tutorial

## Version Check

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.7'

## Connecting

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///:memory:', echo=True)

## Declare a Mapping

In [4]:
from sqlalchemy.ext.declarative import declarative_base

In [5]:
Base = declarative_base()

In [6]:
from sqlalchemy import Column, Integer, String

In [7]:
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return "<User(name='{}', fullname='{}', nickname='{}')>".format(
        self.name, self.fullname, self.nickname)

## Create a Schema

In [8]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [9]:
Base.metadata.create_all(engine)

2019-09-27 11:40:55,323 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-09-27 11:40:55,326 INFO sqlalchemy.engine.base.Engine ()
2019-09-27 11:40:55,328 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-09-27 11:40:55,328 INFO sqlalchemy.engine.base.Engine ()
2019-09-27 11:40:55,330 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-09-27 11:40:55,331 INFO sqlalchemy.engine.base.Engine ()
2019-09-27 11:40:55,332 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2019-09-27 11:40:55,332 INFO sqlalchemy.engine.base.Engine ()
2019-09-27 11:40:55,334 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-09-27 11:40:55,335 INFO sqlalchemy.engine.base.Engine ()
2019-09-27 11:40:55,336 INFO sqlalchemy.engine.base.Engine COMMIT


## Create an Instance of the Mapped Class

In [10]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='ednickname')

## Creating a Session

In [11]:
from sqlalchemy.orm import sessionmaker

In [12]:
Session = sessionmaker(bind=engine)

In [13]:
session = Session()

## Adding and Updating Objects

In [14]:
session.add(ed_user)

In [15]:
our_user = session.query(User).filter_by(name='ed').first()

2019-09-27 11:47:08,675 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-27 11:47:08,677 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-27 11:47:08,678 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'ednickname')
2019-09-27 11:47:08,680 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-09-27 11:47:08,681 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [16]:
ed_user is our_user

True

In [17]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')
])

In [18]:
ed_user.nickname = 'eddie'

In [19]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [20]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [21]:
session.commit()

2019-09-27 11:53:46,310 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2019-09-27 11:53:46,311 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2019-09-27 11:53:46,312 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-27 11:53:46,312 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2019-09-27 11:53:46,313 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-27 11:53:46,314 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2019-09-27 11:53:46,315 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-27 11:53:46,315 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2019-09-27 11:53:46,316 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
ed_user.id

2019-09-27 11:54:11,465 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-27 11:54:11,466 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-09-27 11:54:11,466 INFO sqlalchemy.engine.base.Engine (1,)


1

## Rolling Back

In [23]:
ed_user.name = 'Edwardo'

In [24]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [25]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2019-09-27 11:57:58,110 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2019-09-27 11:57:58,111 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2019-09-27 11:57:58,112 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-09-27 11:57:58,113 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2019-09-27 11:57:58,115 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2019-09-27 11:57:58,115 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [26]:
session.rollback()

2019-09-27 13:50:25,065 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [27]:
ed_user.name

2019-09-27 13:50:35,531 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-27 13:50:35,533 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-09-27 13:50:35,534 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [28]:
fake_user in session

False

In [29]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2019-09-27 13:52:14,250 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2019-09-27 13:52:14,251 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

## Querying

In [30]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2019-09-27 14:16:14,108 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2019-09-27 14:16:14,109 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [31]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2019-09-27 14:18:34,250 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-09-27 14:18:34,251 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [32]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2019-09-27 14:19:58,935 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2019-09-27 14:19:58,936 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', nickname='eddie')> ed
<User(name='wendy', fullname='Wendy Williams', nickname='windy')> wendy
<User(name='mary', fullname='Mary Contrary', nickname='mary')> mary
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')> fred


In [33]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2019-09-27 14:21:46,615 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2019-09-27 14:21:46,616 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


In [34]:
from sqlalchemy.orm import aliased

In [35]:
user_alias = aliased(User, name='user_alias')

In [36]:
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2019-09-27 14:24:16,829 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.nickname AS user_alias_nickname 
FROM users AS user_alias
2019-09-27 14:24:16,830 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', nickname='eddie')>
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>


In [37]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2019-09-27 14:26:24,293 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2019-09-27 14:26:24,294 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>


In [42]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2019-09-27 14:30:10,235 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-09-27 14:30:10,236 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


In [43]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2019-09-27 14:30:21,731 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-09-27 14:30:21,732 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


In [44]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2019-09-27 14:32:13,853 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2019-09-27 14:32:13,854 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name='ed', fullname='Ed Jones', nickname='eddie')>


## Returning Lists and Scalars

In [46]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)
query.all()

2019-09-27 14:38:27,629 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-09-27 14:38:27,630 INFO sqlalchemy.engine.base.Engine ('%ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

In [47]:
query.first()

2019-09-27 14:39:42,155 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2019-09-27 14:39:42,156 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='eddie')>

In [48]:
user = query.one()

2019-09-27 14:40:29,852 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-09-27 14:40:29,853 INFO sqlalchemy.engine.base.Engine ('%ed',)


MultipleResultsFound: Multiple rows were found for one()

In [49]:
query = session.query(User.id).filter(User.name == 'ed').order_by(User.id)

In [50]:
query.scalar()

2019-09-27 14:42:50,629 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id 
FROM users 
WHERE users.name = ? ORDER BY users.id
2019-09-27 14:42:50,630 INFO sqlalchemy.engine.base.Engine ('ed',)


1

## Using Textual SQL